Датасет содержит информацию о продаже билетов авиакомпании. Каждая строка - уникальная покупка.
ISSUE_DATE	дата покупки
FLIGHT_DATE_LOC	дата совершения перелета
PAX_TYPE	тип пассажиров
REVENUE_AMOUNT	сумма (значения скорректированы, но пропорции сохранены)
ORIG_CITY_CODE	город отправления
DEST_CITY_CODE	город назначения
ROUTE_FLIGHT_TYPE	тип перелета
FFP_FLAG	наличие программы лояльности
SALE_TYPE	способ покупки

Расшифровка значений в полях:
PAX_TYPE	
AD	ADULT
CHD	CHILD
INF	UNDEFINED

ROUTE_FLIGHT_TYPE	
ВВЛ	Внутренние Воздушные Линии
МВЛ	Международные Воздушные Линии
	
Дополнительная информация
данные по кодам аэропортов
https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
список полей:
column_names = ['Airport Name', 'Location', 'Country', 'IATA_Code', 'ICAO_Code', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Timezone Region', 'Type', 'Source']


1. Познакомьтесь с данными.
2. Осуществите предобработку данных.
    - 2.1 Приведите заголовки к удобному виду.
    - 2.2 Проверьте типы данных.
    - 2.3 Обработайте пропуски.
    - 2.4 Проверьте наличие дубликатов.
    - 2.5 Исключите нулевые значения в выручке для дальнейшего исследования. 
3. Исследовательский анализ данных:
     - 3.1 Создайте отчет о динамике продаж (выручке). Постройте линейный график. Повторите отчет с детализацией по полям: FFP_FLAG, ROUTE_FLIGHT_TYPE, SALE_TYPE. Постройте общие отчеты по выручке по типам пассажиров, по типам перелета, по способам покупки, по программам лояльности. Визуализируйте каждый отчет. Решение о детализации отчетов примите самостоятельно.
     - 3.2 Исследуйте выручку, примите решение об исключении выбросов для дальнейшего исследования, руководствуясь минимизацией потерь в данных.
     - 3.3 Исследуйте сезонность продаж. Дополнительно проверьте гипотезу о том, что выручка от продаж осенью и зимой отличается. Исследуйте выручку и число продаж по месяцам, проверьте корреляцию.
     - 3.4 Клиенты покупают билеты в разное время. Кто-то покупает билеты заранее, другие - в последний момент. Проверьте гипотезу о том, что выручка от продажи билетов для этих двух групп клиентов отличается. Предварительно разбейте клиентов минимум на три группы.
     - 3 .5  Определите топ-10 популярных направлений перелета (город-город) отдельно для ВМЛ и МВЛ. Изучите выручку для этих направлений. Для этого добавьте в датасет сведения о городах, используя данные из датасета:
     https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
     - 3.6 Определите есть ли зависимость выручки от других признаков. Постройте матрицу корреляций.

Каждый пункт исследования сопроводите выводами.


In [109]:
# импортируем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import t, norm
from scipy import stats as st

In [110]:
data = pd.read_csv('air_data.csv', sep=';')

In [111]:
data.head()

,ISSUE_DATE,FLIGHT_DATE_LOC,PAX_TYPE,REVENUE_AMOUNT,ORIG_CITY_CODE,DEST_CITY_CODE,ROUTE_FLIGHT_TYPE,FFP_FLAG,SALE_TYPE
0,2021-12-07 00:00:00,2022-01-11 00:00:00,AD,200,MOW,KGD,ВВЛ,NaN,ONLINE
1,2021-12-31 00:00:00,2022-01-03 00:00:00,AD,610,UUS,OVB,ВВЛ,NaN,OFFLINE
2,2021-11-28 00:00:00,2022-01-04 00:00:00,AD,390,KGD,MOW,ВВЛ,FFP,ONLINE
3,2021-12-08 00:00:00,2022-01-05 00:00:00,AD,190,VOZ,MOW,ВВЛ,NaN,ONLINE
4,2021-12-08 00:00:00,2022-01-12 00:00:00,AD,190,MOW,KRR,ВВЛ,FFP,OFFLINE


In [112]:
data.columns = data.columns.str.lower()

In [113]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   issue_date         50000 non-null  object
 1   flight_date_loc    50000 non-null  object
 2   pax_type           50000 non-null  object
 3   revenue_amount     50000 non-null  int64 
 4   orig_city_code     49998 non-null  object
 5   dest_city_code     49999 non-null  object
 6   route_flight_type  50000 non-null  object
 7   ffp_flag           19398 non-null  object
 8   sale_type          50000 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.4+ MB


In [114]:
data['issue_date'] = pd.to_datetime(data['issue_date'], format='%Y-%m-%d %H:%M:%S')
data['flight_date_loc'] = pd.to_datetime(data['flight_date_loc'], format='%Y-%m-%d %H:%M:%S')

In [115]:
data['revenue_amount'] = data['revenue_amount'].astype('float64')

In [116]:
data.isna().sum()

issue_date               0
flight_date_loc          0
pax_type                 0
revenue_amount           0
orig_city_code           2
dest_city_code           1
route_flight_type        0
ffp_flag             30602
sale_type                0
dtype: int64

In [117]:
data[data['orig_city_code'].isna() == True]

,issue_date,flight_date_loc,pax_type,revenue_amount,orig_city_code,dest_city_code,route_flight_type,ffp_flag,sale_type
2020,2022-02-17,2022-03-22,AD,440.0,NaN,KHV,МВЛ,NaN,ONLINE
8100,2022-05-02,2022-05-11,AD,440.0,NaN,KHV,МВЛ,NaN,OFFLINE


In [118]:
data[data['dest_city_code'].isna() == True]

,issue_date,flight_date_loc,pax_type,revenue_amount,orig_city_code,dest_city_code,route_flight_type,ffp_flag,sale_type
47582,2022-05-13,2022-07-28,AD,440.0,KHV,NaN,МВЛ,NaN,OFFLINE


In [119]:
data['ffp_flag'].unique()

array([nan, 'FFP'], dtype=object)

In [120]:
data = data.dropna(subset=['orig_city_code'])
data = data.dropna(subset=['dest_city_code'])
data['ffp_flag'] = data['ffp_flag'].fillna('NOT')

In [121]:
data.isna().sum()

issue_date           0
flight_date_loc      0
pax_type             0
revenue_amount       0
orig_city_code       0
dest_city_code       0
route_flight_type    0
ffp_flag             0
sale_type            0
dtype: int64

In [122]:
data.drop(data.query('revenue_amount == 0').index, inplace = True)

In [123]:
data.reset_index()

,index,issue_date,flight_date_loc,pax_type,revenue_amount,orig_city_code,dest_city_code,route_flight_type,ffp_flag,sale_type
0,0,2021-12-07,2022-01-11,AD,200.0,MOW,KGD,ВВЛ,NOT,ONLINE
1,1,2021-12-31,2022-01-03,AD,610.0,UUS,OVB,ВВЛ,NOT,OFFLINE
2,2,2021-11-28,2022-01-04,AD,390.0,KGD,MOW,ВВЛ,FFP,ONLINE
3,3,2021-12-08,2022-01-05,AD,190.0,VOZ,MOW,ВВЛ,NOT,ONLINE
4,4,2021-12-08,2022-01-12,AD,190.0,MOW,KRR,ВВЛ,FFP,OFFLINE
...,...,...,...,...,...,...,...,...,...,...
49526,49995,2022-06-11,2022-06-22,CHD,480.0,MOW,OMS,ВВЛ,NOT,ONLINE
49527,49996,2022-06-06,2022-06-20,AD,270.0,LED,MOW,ВВЛ,NOT,OFFLINE
49528,49997,2022-06-26,2022-06-29,AD,230.0,MOW,KZN,ВВЛ,FFP,ONLINE
49529,49998,2022-06-26,2022-06-28,AD,270.0,KZN,MOW,ВВЛ,NOT,OFFLINE
